In [ ]:
from llama_index.core.memory import (
    VectorMemory,
    SimpleComposableMemory,
    ChatMemoryBuffer,
)
from llama_index.embeddings.openai import OpenAIEmbedding

vector_memory = VectorMemory.from_defaults(
    vector_store=None,  # leave as None to use default in-memory vector store
    embed_model=OpenAIEmbedding(),
    retriever_kwargs={"similarity_top_k": 1},
)

chat_memory_buffer = ChatMemoryBuffer.from_defaults()

composable_memory = SimpleComposableMemory.from_defaults(
    sources=[chat_memory_buffer, vector_memory]
)

In [ ]:
from llama_index.core.llms import ChatMessage

msgs = [
    ChatMessage.from_str("Jerry likes juice.", "user"),
    ChatMessage.from_str("Bob likes burgers.", "user"),
    ChatMessage.from_str("Alice likes apples.", "user"),
]

In [ ]:
# load into memory
for m in msgs:
    composable_memory.put(m)

In [ ]:
# retrieve from memory
msgs = composable_memory.get("What does Jerry like?")
msgs

[ChatMessage(role=<MessageRole.USER: 'user'>, content='Jerry likes juice.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Bob likes burgers.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Alice likes apples.', additional_kwargs={})]

In [ ]:
msgs = composable_memory.get_and_set("What does Jerry like?")
msgs

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are a helpful assistant.\n\nBelow are a set of relevant dialogues retrieved from potentially several memory sources:\n\n=====Relevant messages from memory source 1=====\n\n\tUSER: Jerry likes juice.\n\n=====End of relevant messages from memory source 1======\n\nThis is the end of the of retrieved message dialogues.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Jerry likes juice.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Bob likes burgers.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Alice likes apples.', additional_kwargs={})]

In [ ]:
composable_memory.reset()

In [ ]:
msgs = [
    ChatMessage.from_str("Jerry likes burgers.", "user"),
    ChatMessage.from_str("Bob likes apples.", "user"),
    ChatMessage.from_str("Indeed, Bob likes apples.", "assistant"),
    ChatMessage.from_str("Alice likes juice.", "user"),
]
composable_memory.set(msgs)

In [ ]:
msgs = composable_memory.get_and_set("What does Bob like?")
msgs

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are a helpful assistant.\n\nBelow are a set of relevant dialogues retrieved from potentially several memory sources:\n\n=====Relevant messages from memory source 1=====\n\n\tUSER: Bob likes apples.\n\tASSISTANT: Indeed, Bob likes apples.\n\n=====End of relevant messages from memory source 1======\n\nThis is the end of the of retrieved message dialogues.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Jerry likes burgers.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Bob likes apples.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Indeed, Bob likes apples.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Alice likes juice.', additional_kwargs={})]

In [ ]:
print(msgs[0].content)

You are a helpful assistant.

Below are a set of relevant dialogues retrieved from potentially several memory sources:

=====Relevant messages from memory source 1=====

	USER: Bob likes apples.
	ASSISTANT: Indeed, Bob likes apples.

=====End of relevant messages from memory source 1======

This is the end of the of retrieved message dialogues.


### Plug Memory Module Into Agent

In [ ]:
vector_memory = VectorMemory.from_defaults(
    vector_store=None,  # leave as None to use default in-memory vector store
    embed_model=OpenAIEmbedding(),
    retriever_kwargs={"similarity_top_k": 1},
)

chat_memory_buffer = ChatMemoryBuffer.from_defaults()

composable_memory = SimpleComposableMemory.from_defaults(
    sources=[chat_memory_buffer, vector_memory]
)

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker

import nest_asyncio

nest_asyncio.apply()

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def mystery(a: int, b: int) -> int:
    """Mystery function on two numbers"""
    return a**2 - b**2


multiply_tool = FunctionTool.from_defaults(fn=multiply)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, mystery_tool], llm=llm, verbose=True
)
agent = agent_worker.as_agent(memory=composable_memory)

In [ ]:
response = agent.chat("What is the mystery function on 5 and 6?")
print(str(response))

Added user message to memory: What is the mystery function on 5 and 6?
=== Calling Function ===
Calling function: mystery with args: {"a": 5, "b": 6}
=== Function Output ===
-11
=== LLM Response ===
The mystery function on 5 and 6 returns -11.
assistant: The mystery function on 5 and 6 returns -11.


In [ ]:
agent.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the mystery function on 5 and 6?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [ChatCompletionMessageToolCall(id='call_ffPgUmNuVavvvDvmHhslNCx2', function=Function(arguments='{\n  "a": 5,\n  "b": 6\n}', name='mystery'), type='function')]}),
 ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='-11', additional_kwargs={'name': 'mystery', 'tool_call_id': 'call_ffPgUmNuVavvvDvmHhslNCx2'}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The mystery function on 5 and 6 returns -11.', additional_kwargs={})]

In [ ]:
agent.memory.reset()

In [ ]:
agent.chat_history

[]

In [ ]:
response = agent.chat(
    "What was the output of the mystery function on 5 and 6 again? Don't recompute."
)

Added user message to memory: What was the output of the mystery function on 5 and 6 again? Don't recompute.
=== LLM Response ===
I'm sorry, but I don't have access to the previous output of the mystery function on 5 and 6.


In [ ]:
# retrieve from long term
agent.memory.get_and_set("mystery")

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are a helpful assistant.\n\nBelow are a set of relevant dialogues retrieved from potentially several memory sources:\n\n=====Relevant messages from memory source 1=====\n\n\tUSER: What is the mystery function on 5 and 6?\n\tASSISTANT: None\n\tTOOL: -11\n\tASSISTANT: The mystery function on 5 and 6 returns -11.\n\n=====End of relevant messages from memory source 1======\n\nThis is the end of the of retrieved message dialogues.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content="What was the output of the mystery function on 5 and 6 again? Don't recompute.", additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="I'm sorry, but I don't have access to the previous output of the mystery function on 5 and 6.", additional_kwargs={})]

In [ ]:
response = agent.chat(
    "What was the output of the mystery function on 5 and 6 again? Don't recompute."
)

Added user message to memory: What was the output of the mystery function on 5 and 6 again? Don't recompute.
=== LLM Response ===
The output of the mystery function on 5 and 6 is -11.
